Paper: https://arxiv.org/abs/1506.00196

### setup

In [1]:
import argparse
import os
import time

In [2]:
import Levenshtein  # to compute phoneme error rate

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.nn.utils import clip_grad_norm
import torchtext.data as data

In [4]:
parser = {
    'data_path': '../data/cmudict/',
    'epochs': 50,
    'batch_size': 100,
    'max_len': 20,
    'beam_size': 3,
    'd_embed': 500,
    'd_hidden': 500,
    'attention': True,
    'log_every': 100,
    'lr': 0.007,
    'lr_decay': 0.5,
    'lr_min': 1e-5,  # stop when lr is too low
    'n_bad_loss': 5,  # number of bad loss before decaying
    'clip': 2.3,  # clip gradient
    'cuda': True,
    'seed': 5,
    'intermediate_path': '../intermediate/g2p/',
}
args = argparse.Namespace(**parser)

In [5]:
args.cuda = args.cuda and torch.cuda.is_available()

if not os.path.isdir(args.intermediate_path):
    os.makedirs(args.intermediate_path)
if not os.path.isdir(args.data_path):
    URL = "https://github.com/cmusphinx/cmudict/archive/master.zip"
    !wget $URL -O ../data/cmudict.zip
    !unzip ../data/cmudict.zip -d ../data/
    !mv ../data/cmudict-master $args.data_path

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

### model

In [6]:
class Encoder(nn.Module):

    def __init__(self, vocab_size, d_embed, d_hidden):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_embed)
        self.lstm = nn.LSTMCell(d_embed, d_hidden)
        self.d_hidden = d_hidden

    def forward(self, x_seq, cuda=False):
        o = []
        # dim(e_seq): len_seq x batch_size x d_embed
        e_seq = self.embedding(x_seq)
        tt = torch.cuda if cuda else torch
        h = Variable(tt.FloatTensor(e_seq.size(1), self.d_hidden).zero_())
        c = Variable(tt.FloatTensor(e_seq.size(1), self.d_hidden).zero_())
        for e in e_seq.chunk(e_seq.size(0), 0):
            e = e.squeeze(0)
            h, c = self.lstm(e, (h, c))
            o.append(h)
        return torch.stack(o, 0), h, c

In [7]:
# based on https://github.com/OpenNMT/OpenNMT-py

class Attention(nn.Module):
    """Dot global attention from https://arxiv.org/abs/1508.04025"""
    def __init__(self, dim):
        super(Attention, self).__init__()
        self.linear = nn.Linear(dim*2, dim, bias=False)
        
    def forward(self, x, context=None):
        if context is None:
            return x
        assert x.size(0) == context.size(0)  # x: batch x dim
        assert x.size(1) == context.size(2)  # context: batch x seq x dim
        attn = F.softmax(context.bmm(x.unsqueeze(2)).squeeze(2))
        weighted_context = attn.unsqueeze(1).bmm(context).squeeze(1)
        o = self.linear(torch.cat((x, weighted_context), 1))
        return F.tanh(o)

In [28]:
class Decoder(nn.Module):
    
    def __init__(self, vocab_size, d_embed, d_hidden):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_embed)
        self.lstm = nn.LSTMCell(d_embed, d_hidden)
        self.attn = Attention(d_hidden)
        self.linear = nn.Linear(d_hidden, vocab_size)

    def forward(self, x_seq, h, c, context=None):
        o = []
        e_seq = self.embedding(x_seq)
        for e in e_seq.chunk(e_seq.size(0), 0):
            e = e.squeeze(0)
            h, c = self.lstm(e, (h, c))
            o.append(self.attn(h, context))
        o = torch.stack(o, 0)
        o = self.linear(o.view(-1, h.size(1)))
        return F.log_softmax(o).view(x_seq.size(0), -1, o.size(1)), h, c

In [27]:
class G2P(nn.Module):
    
    def __init__(self, config):
        super(G2P, self).__init__()
        self.encoder = Encoder(config.g_size, config.d_embed,
                               config.d_hidden)
        self.decoder = Decoder(config.p_size, config.d_embed,
                               config.d_hidden)
        self.config = config
        
    def forward(self, g_seq, p_seq=None):
        o, h, c = self.encoder(g_seq, self.config.cuda)
        context = o.t() if self.config.attention else None
        if p_seq is not None:  # not generate
            return self.decoder(p_seq, h, c, context)
        else:  # start with <os> -> 1
            assert g_seq.size(1) == 1  # make sure batch_size = 1
            return self._generate(h, c, context)
        
    def _generate(self, h, c, context):
        beam = Beam(self.config.beam_size, cuda=self.config.cuda)
        h = h.expand(beam.size, h.size(1))
        c = c.expand(beam.size, c.size(1))
        context = context.expand(beam.size, context.size(1), context.size(2))
        for i in range(self.config.max_len):  # max_len = 20
            x = beam.get_current_state()  # beam to batch
            o, h, c = self.decoder(Variable(x.unsqueeze(0)), h, c, context)
            if beam.advance(o.data.squeeze(0)):
                break
            h.data.copy_(h.data.index_select(0, beam.get_current_origin()))
            c.data.copy_(c.data.index_select(0, beam.get_current_origin()))
        tt = torch.cuda if self.config.cuda else torch
        return Variable(tt.LongTensor(beam.get_hyp(0)))

### utils

In [10]:
# Code borrowed from PyTorch OpenNMT
# https://github.com/OpenNMT/OpenNMT-py/
# https://github.com/MaximumEntropy/Seq2Seq-PyTorch/

class Beam(object):
    """Ordered beam of candidate outputs."""

    def __init__(self, size, pad=1, bos=2, eos=3, cuda=False):
        """Initialize params."""
        self.size = size
        self.done = False
        self.pad = pad
        self.bos = bos
        self.eos = eos
        self.tt = torch.cuda if cuda else torch

        # The score for each translation on the beam.
        self.scores = self.tt.FloatTensor(size).zero_()

        # The backpointers at each time-step.
        self.prevKs = []

        # The outputs at each time-step.
        self.nextYs = [self.tt.LongTensor(size).fill_(self.pad)]
        self.nextYs[0][0] = self.bos

    # Get the outputs for the current timestep.
    def get_current_state(self):
        """Get state of beam."""
        return self.nextYs[-1]

    # Get the backpointers for the current timestep.
    def get_current_origin(self):
        """Get the backpointer to the beam at this step."""
        return self.prevKs[-1]

    def advance(self, workd_lk):
        """Advance the beam."""
        num_words = workd_lk.size(1)

        # Sum the previous scores.
        if len(self.prevKs) > 0:
            beam_lk = workd_lk + self.scores.unsqueeze(1).expand_as(workd_lk)
        else:
            beam_lk = workd_lk[0]

        flat_beam_lk = beam_lk.view(-1)

        bestScores, bestScoresId = flat_beam_lk.topk(self.size, 0,
                                                     True, True)
        self.scores = bestScores

        # bestScoresId is flattened beam x word array, so calculate which
        # word and beam each score came from
        prev_k = bestScoresId / num_words
        self.prevKs.append(prev_k)
        self.nextYs.append(bestScoresId - prev_k * num_words)
        # End condition is when top-of-beam is EOS.
        if self.nextYs[-1][0] == self.eos:
            self.done = True
        return self.done

    def get_hyp(self, k):
        """Get hypotheses."""
        hyp = []
        # print(len(self.prevKs), len(self.nextYs), len(self.attn))
        for j in range(len(self.prevKs) - 1, -1, -1):
            hyp.append(self.nextYs[j + 1][k])
            k = self.prevKs[j][k]
        return hyp[::-1]

In [11]:
class CMUDict(data.Dataset):

    def __init__(self, data_lines, g_field, p_field):
        fields = [('grapheme', g_field), ('phoneme', p_field)]
        examples = []  # maybe ignore '...-1' grapheme
        for line in data_lines:
            grapheme, phoneme = line.split(maxsplit=1)
            examples.append(data.Example.fromlist([grapheme, phoneme],
                                                  fields))
        self.sort_key = lambda x: len(x.grapheme)
        super(CMUDict, self).__init__(examples, fields)
    
    @classmethod
    def splits(cls, path, g_field, p_field, seed=None):
        import random
        
        if seed is not None:
            random.seed(seed)
        with open(path) as f:
            lines = f.readlines()
        random.shuffle(lines)
        train_lines, val_lines, test_lines = [], [], []
        for i, line in enumerate(lines):
            if i % 20 == 0:
                val_lines.append(line)
            elif i % 20 < 3:
                test_lines.append(line)
            else:
                train_lines.append(line)
        train_data = cls(train_lines, g_field, p_field)
        val_data = cls(val_lines, g_field, p_field)
        test_data = cls(test_lines, g_field, p_field)
        return (train_data, val_data, test_data)

In [12]:
# Code borrowed from https://github.com/SeanNaren/deepspeech.pytorch

def phoneme_error_rate(p_seq1, p_seq2):
    p_vocab = set(p_seq1 + p_seq2)
    p2c = dict(zip(p_vocab, range(len(p_vocab))))
    c_seq1 = [chr(p2c[p]) for p in p_seq1]
    c_seq2 = [chr(p2c[p]) for p in p_seq2]
    return Levenshtein.distance(''.join(c_seq1),
                                ''.join(c_seq2)) / len(c_seq2)

In [13]:
def adjust_learning_rate(optimizer, lr_decay):
    for param_group in optimizer.param_groups:
        param_group['lr'] *= lr_decay

### train

In [14]:
def train(config, train_iter, model, criterion, optimizer, epoch):
    global iteration, n_total, train_loss, n_bad_loss
    global init, best_val_loss, stop
    
    print("=> EPOCH {}".format(epoch))
    train_iter.init_epoch()
    for batch in train_iter:
        iteration += 1
        model.train()
        
        output, _, __ = model(batch.grapheme, batch.phoneme[:-1].detach())
        target = batch.phoneme[1:]
        loss = criterion(output.view(output.size(0) * output.size(1), -1),
                         target.view(target.size(0) * target.size(1)))
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), config.clip, 'inf')
        optimizer.step()
        
        n_total += batch.batch_size
        train_loss += loss.data[0] * batch.batch_size
        
        if iteration % config.log_every == 0:
            train_loss /= n_total
            val_loss = validate(val_iter, model, criterion)
            print("   % Time: {:5.0f} | Iteration: {:5} | Batch: {:4}/{}"
                  " | Train loss: {:.4f} | Val loss: {:.4f}"
                  .format(time.time()-init, iteration, train_iter.iterations,
                          len(train_iter), train_loss, val_loss))
            
            n_total = train_loss = 0
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                n_bad_loss = 0
                torch.save(model.state_dict(), config.best_model)
            else:
                n_bad_loss += 1
            if n_bad_loss == config.n_bad_loss:
                adjust_learning_rate(optimizer, config.lr_decay)
                new_lr = optimizer.param_groups[0]['lr']
                print("=> Adjust learning rate to: {}".format(new_lr))
                if new_lr < config.lr_min:
                    stop = True
                    break
                n_bad_loss = 0

In [15]:
def validate(val_iter, model, criterion):
    model.eval()
    val_loss = 0
    val_iter.init_epoch()
    for batch in val_iter:
        output, _, __ = model(batch.grapheme, batch.phoneme[:-1])
        target = batch.phoneme[1:]
        loss = criterion(output.squeeze(1), target.squeeze(1))
        val_loss += loss.data[0] * batch.batch_size
    return val_loss / len(val_iter.dataset)

In [16]:
def test(test_iter, model, criterion):
    model.eval()
    test_iter.init_epoch()
    test_per = test_wer = 0
    for batch in test_iter:
        output = model(batch.grapheme).data.tolist()
        target = batch.phoneme[1:].squeeze(1).data.tolist()
        # calculate per, wer here
        per = phoneme_error_rate(output, target) 
        wer = int(output != target)
        test_per += per  # batch_size = 1
        test_wer += wer
        
    test_per = test_per / len(test_iter.dataset) * 100
    test_wer = test_wer / len(test_iter.dataset) * 100
    print("Phoneme error rate (PER): {:.2f}\nWord error rate (WER): {:.2f}"
          .format(test_per, test_wer))

In [17]:
def show(batch, model):
    assert batch.batch_size == 1
    g_field = batch.dataset.fields['grapheme']
    p_field = batch.dataset.fields['phoneme']
    prediction = model(batch.grapheme).data.tolist()[:-1]
    grapheme = batch.grapheme.squeeze(1).data.tolist()[1:][::-1]
    phoneme = batch.phoneme.squeeze(1).data.tolist()[1:-1]
    print("> {}\n= {}\n< {}\n".format(
        ''.join([g_field.vocab.itos[g] for g in grapheme]),
        ' '.join([p_field.vocab.itos[p] for p in phoneme]),
        ' '.join([p_field.vocab.itos[p] for p in prediction])))

### prepare

In [18]:
g_field = data.Field(init_token='<s>',
                     tokenize=(lambda x: list(x.split('(')[0])[::-1]))
p_field = data.Field(init_token='<os>', eos_token='</os>',
                     tokenize=(lambda x: x.split('#')[0].split()))

In [19]:
filepath = os.path.join(args.data_path, 'cmudict.dict')
train_data, val_data, test_data = CMUDict.splits(filepath, g_field, p_field,
                                                 args.seed)

In [20]:
g_field.build_vocab(train_data, val_data, test_data)
p_field.build_vocab(train_data, val_data, test_data)

In [21]:
device = None if args.cuda else -1  # None is current gpu
train_iter = data.BucketIterator(train_data, batch_size=args.batch_size,
                                 repeat=False, device=device)
val_iter = data.Iterator(val_data, batch_size=1,
                         train=False, sort=False, device=device)
test_iter = data.Iterator(test_data, batch_size=1,
                          train=False, shuffle=True, device=device)

In [29]:
config = args
config.g_size = len(g_field.vocab)
config.p_size = len(p_field.vocab)
config.best_model = os.path.join(config.intermediate_path,
                                 "best_model_adagrad_attn.pth")

model = G2P(config)
criterion = nn.NLLLoss()
if config.cuda:
    model.cuda()
    criterion.cuda()
optimizer = optim.Adagrad(model.parameters(), lr=config.lr)  # use Adagrad

### run

In [23]:
if 1 == 1:  # change to True to train
    iteration = n_total = train_loss = n_bad_loss = 0
    stop = False
    best_val_loss = 10
    init = time.time()
    for epoch in range(1, config.epochs+1):
        train(config, train_iter, model, criterion, optimizer, epoch)
        if stop:
            break

=> EPOCH 1
   % Time:   107 | Iteration:   100 | Batch:  100/1148 | Train loss: 1.2808 | Val loss: 0.8004
   % Time:   213 | Iteration:   200 | Batch:  200/1148 | Train loss: 0.5093 | Val loss: 0.5768
   % Time:   320 | Iteration:   300 | Batch:  300/1148 | Train loss: 0.4187 | Val loss: 0.5115
   % Time:   426 | Iteration:   400 | Batch:  400/1148 | Train loss: 0.3665 | Val loss: 0.4574
   % Time:   533 | Iteration:   500 | Batch:  500/1148 | Train loss: 0.3454 | Val loss: 0.4275
   % Time:   639 | Iteration:   600 | Batch:  600/1148 | Train loss: 0.3197 | Val loss: 0.4042
   % Time:   745 | Iteration:   700 | Batch:  700/1148 | Train loss: 0.3122 | Val loss: 0.3906
   % Time:   852 | Iteration:   800 | Batch:  800/1148 | Train loss: 0.3087 | Val loss: 0.3916
   % Time:   959 | Iteration:   900 | Batch:  900/1148 | Train loss: 0.2912 | Val loss: 0.3749
   % Time:  1065 | Iteration:  1000 | Batch: 1000/1148 | Train loss: 0.2893 | Val loss: 0.3565
   % Time:  1172 | Iteration:  1100 | B

### test

In [30]:
model.load_state_dict(torch.load(config.best_model))
test(test_iter, model, criterion)

Phoneme error rate (PER): 9.13
Word error rate (WER): 39.78


In [31]:
test_iter.init_epoch()
for i, batch in enumerate(test_iter):
    show(batch, model)
    if i == 10:
        break

> conclude
= K AH0 N K L UW1 D
< K AH0 N K L UW1 D

> chafe
= CH EY1 F
< CH EY1 F

> methanex
= M EH1 TH AH0 N EH2 K S
< M EH1 TH AH0 N EH2 K S

> contrariness
= K AA1 N T R EH0 R IY0 N AH0 S
< K AH0 N T R EH1 R IY0 N AH0 S

> majcher
= M AE1 JH K ER0
< M AE1 JH K ER0

> geac
= JH IY1 IY1 EY1 S IY1
< G IY1 K

> cholesterol
= K AH0 L EH1 S T ER0 AH0 L
< K AH0 L EH1 S T ER0 AO2 L

> lawes
= L AO1 Z
< L AO1 Z

> encyclopedias
= IH0 N S AY2 K L AH0 P IY1 D IY0 AH0 Z
< IH0 N S AY2 K L AH0 P IY1 D IY0 AH0 Z

> pedantry
= P EH1 D AH0 N T R IY0
< P EH1 D AH0 N T R IY0

> wale
= W EY1 L
< W EY1 L

